# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [31]:
import os, requests
import random
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import numpy as np
import math

In [30]:
API_KEY = os.environ.get('key')

In [40]:
load_dotenv()
REGION = 'na1'
root = 'https://americas.api.riotgames.com'
winRateDict = {}
pickRateDict = {}
top5WR = {}

champList = ['aphelios', 'ashe', 'caitlyn', 'corki', 'draven', 'ezreal', 'jhin', 'jinx', 'kaisa', 'kalista', 'kogmaw', 'lucian', 'missfortune', 'nilah', 'samira', 'sivir', 'smolder', 'tristana', 'twitch', 'varus', 'vayne', 'xayah', 'zeri']
champListLength = len(champList)
headersOPGG = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}

In [38]:
tierSort = '?tier=master_plus' #NOTE: Edit as needed

In [58]:
def updateWinRates(winRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build/{tierSort}'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            winRateEm = soup.find('em', string='Win rate')
            winRateB = winRateEm.find_next('b', class_='text-[12px]')
            if winRateB:
                raw_text = winRateB.get_text(separator=" ", strip=True)
                winRate = raw_text.replace('"', '').strip()
                winRateDict.update({champ: float(winRate.strip(" %"))})
        else:
            print(f"Failed to retrieve data: {response.status_code}")

def updatePickRates(pickRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build/{tierSort}'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            pickRateEm = soup.find('em', string='Pick rate')
            pickRateB = pickRateEm.find_next('b', class_='text-[12px]')
            if pickRateB:
                raw_text = pickRateB.get_text(separator=" ", strip=True)
                pickRate = raw_text.replace('"', '').strip()
                pickRateDict.update({champ: float(pickRate.strip(" %"))})
        else:
            print(f"Failed to retrieve data: {response.status_code}")

def getTop5(top5, winRateDict):
    for key, value in winRateDict.items():
        currWR = float(winRateDict.get(key))
        if len(top5) < 5:
            top5[key] = currWR
        else:
            smallest_key = min(top5, key=top5.get)
            if currWR > top5[smallest_key]:
                top5.pop(smallest_key)
                top5[key] = currWR

def addZScores(dictionary):
    zScoreDict = {}
    std = np.std(list(dictionary.values()))
    mean = np.mean(list(dictionary.values()))
    for key, value in dictionary.items():
        floatVal = float(value)
        currentKey = key
        zScore = float((floatVal - mean) / std)
        zScoreDict[key] = zScore
    return zScoreDict

def joinZPicKRate(winRateDict, zscoreDict):
    result = {}
    for key, value in winRateDict.items():
        result.update({key: 1.5 * value + 2 * zscoreDict[key]})
    return result


def getMatchHistory(count, username, tagline, headers):
    matches = requests.get(f'{root}/lol/match/v5/matches/by-puuid/{playerPUUID}/ids?start=0&count={count}&api_key={API_KEY}', headers=headers).json()
    return matches

def getMatchInformation(matchID, headers):
    return requests.get(f'{root}/lol/match/v5/matches/{matchID}?api_key={API_KEY}', headers=headers).json()

In [45]:
updateWinRates(winRateDict, headersOPGG)
updatePickRates(pickRateDict, headersOPGG)

In [46]:
getTop5(top5WR, winRateDict)

In [47]:
getTop5(top5WR, winRateDict)

In [48]:
zScorePRDict = addZScores(pickRateDict)
zScoreWRDict = addZScores(winRateDict)

In [49]:
truePowerLevelDict = joinZPicKRate(zScoreWRDict, zScorePRDict)

top5PL = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True)[:5])
truePowerLevelDict = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True))
sortedWR = dict(sorted(winRateDict.items(), key=lambda item: item[1], reverse=True))
sortedPR = dict(sorted(pickRateDict.items(), key=lambda item: item[1], reverse=True))

print(f'Win Rates: {sortedWR}\n')
print(f'Pick Rates: {sortedPR}\n')
print(f'Power Level: {truePowerLevelDict}\n')
print(f'Top 5: {top5PL}\n')

Win Rates: {'nilah': 53.31, 'kogmaw': 53.27, 'tristana': 52.15, 'jinx': 51.91, 'draven': 51.73, 'vayne': 51.53, 'missfortune': 51.47, 'corki': 51.37, 'lucian': 51.06, 'ashe': 50.95, 'twitch': 50.63, 'varus': 50.56, 'xayah': 50.51, 'zeri': 50.44, 'aphelios': 50.27, 'kaisa': 50.23, 'samira': 50.02, 'jhin': 49.85, 'smolder': 49.74, 'sivir': 49.7, 'caitlyn': 49.22, 'kalista': 49.03, 'ezreal': 48.47}

Pick Rates: {'kaisa': 24.43, 'ezreal': 20.5, 'jhin': 16.79, 'tristana': 13.74, 'jinx': 13.41, 'varus': 10.59, 'lucian': 10.04, 'ashe': 7.9, 'caitlyn': 7.41, 'corki': 6.54, 'aphelios': 6.09, 'missfortune': 5.75, 'draven': 5.54, 'xayah': 5.05, 'smolder': 4.33, 'zeri': 4.18, 'twitch': 3.82, 'sivir': 3.63, 'vayne': 3.57, 'kalista': 2.63, 'samira': 2.6, 'kogmaw': 2.03, 'nilah': 1.93}

Power Level: {'kaisa': 4.879181901587604, 'tristana': 3.679156743102831, 'jinx': 3.270064305873314, 'jhin': 1.8432374725210499, 'ezreal': 1.3725034310934525, 'nilah': 1.1585066074679782, 'kogmaw': 1.1423334259489621, 

In [50]:
try:
    username, tagline = input("Enter username and tagline (e.g. Saico#Slide): ").split("#")
    print(f"Username: {username}, Tagline: {tagline}")
except ValueError:
    print("Invalid entry method, try again.\n")

Username: Saico, Tagline: Slide


In [56]:
def getPUUID(username, tagline, headers):
    return requests.get(f'{root}/riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={API_KEY}', headers=headers).json()['puuid']

In [57]:
playerPUUID = getPUUID(username, tagline, headers)
count = 20

matchList = getMatchHistory(count, username, tagline)
print(matchList)

['NA1_5257793042', 'NA1_5257769139', 'NA1_5257751614', 'NA1_5257733241', 'NA1_5257661308', 'NA1_5257625478', 'NA1_5257614615', 'NA1_5257304145', 'NA1_5257282077', 'NA1_5257187868', 'NA1_5257170379', 'NA1_5257158710', 'NA1_5257147580', 'NA1_5256858062', 'NA1_5256837726', 'NA1_5256830246', 'NA1_5256819084', 'NA1_5255639416', 'NA1_5255620896', 'NA1_5255598057']


In [ ]:
for game in matchList:
    matchInfo = getMatchInformation(game)
    info = matchInfo['info']
    queueID = info['queueId'] #Ranked is 420
    if queueID != 420:
        continue
    metadata = matchInfo['metadata']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(playerPUUID)]
    playerTeam = player['teamId']
    matchID = metadata['matchId']
    gameStartTime = info['gameStartTimestamp']
    gameEndTime = info['gameEndTimestamp']
    for team in teams:
        if team['teamId'] == playerTeam:
            objs = team['objectives']
            baron = objs['baron']
            dragon = objs['dragon']
            # grubs = objs['grubs'] Not available :(
            herald = objs['riftHerald']
            tower = objs['tower']
            inhibitor = objs['inhibitor']
            champion = objs['champion']
    for obj in [baron, dragon, herald, tower, inhibitor]:
        first = obj['first']
        objKills = obj['kills']
    assists = player['assists']
    champLevel = player['champLevel']
    champName = player['championName']
    damageToBuildings = player['damageDealtToBuildings']
    deaths = player['deaths']
    doublekills = player['doubleKills']
    firstBloodAssist = player['firstBloodAssist']
    firstBloodKill = player['firstBloodKill']
    firstTowerAssist = player['firstTowerAssist']
    firstTowerKill = player['firstTowerKill']
    gold = player['goldEarned']
    kills = player['kills']
    longestTimeLiving = player['longestTimeSpentLiving']
    neutralMinionsKilled = player['neutralMinionsKilled']
    minionsKilled = player['totalMinionsKilled']
    turretTakedowns = player['turretTakedowns']
    visionScore = player['visionScore']
    win = player['win']
    wardsPlaced = player['wardsPlaced']
    wardsKilled = player['wardsKilled']
    ctrlWardsBought = player['visionWardsBoughtInGame']